In [ ]:
# put all hists for a beta value on the same plot - real as black, different betas diff colors
# cell+nuc area, maybe nuc/cell ratio, maybe another feature

In [4]:
!pip install pingouin

Looking in indexes: https://artifactory.corp.alleninstitute.org/artifactory/api/pypi/pypi-virtual/simple
    100% |████████████████████████████████| 215kB 706kB/s eta 0:00:01
    100% |████████████████████████████████| 26.1MB 404kB/s eta 0:00:01    76% |████████████████████████▌       | 19.9MB 65.1MB/s eta 0:00:01
  Using cached https://artifactory.corp.alleninstitute.org/artifactory/api/pypi/pypi-virtual/packages/packages/52/3f/f6a428599e0d4497e1595030965b5ba455fd8ade6e977e3c819973c4b41d/pandas-0.25.3-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 13.1MB 608kB/s eta 0:00:01
    100% |████████████████████████████████| 8.7MB 1.0MB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 1.1MB/s eta 0:00:01
    100% |████████████████████████████████| 634kB 1.4MB/s eta 0:00:01
  Stored in directory: /home/gregj/.cache/pip/wheels/0d/26/52/ef291f0a0d80a4f637f706dc597cce4a2858502609e8c3fbc9
  Stored in directory: /home/gregj/.cache/pip/wheels/31/87/b7/e40

In [1]:
import pickle
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import altair as alt
from altair import expr, datum
alt.data_transformers.enable('data_server', port=11112)

ModuleNotFoundError: No module named 'altair'

In [ ]:
# day_18_color = "#0098EA"
# day_32_color = "#FFB2FF"
# day_colors = [day_18_color, day_32_color]

# day_color_scale = alt.Scale(range=day_colors)

In [ ]:
all_feats_save_path = '/allen/aics/modeling/gregj/results/integrated_cell//test_cbvae_beta_ref/results/feats//all_feats.pkl'
intensity_norms = ['unnormalized', 'normalized']
with open(all_feats_save_path, "rb") as f:
    feature_dict = pickle.load(f) 
for i, intensity_norm in enumerate(intensity_norms):
    #features for a specified intensity norm
    feature_dict[i]
    #each sub-dict has real and gen features
    real_feats = feature_dict[i]['real']
    feature_dict[i]['gen']
    #then gen feature dict has features for many betas
    betas = [k for k in feature_dict[i]['gen']]
    features_for_the_first_beta = feature_dict[i]['gen'][betas[0]]
    #the real columns and the generated columns (for each beta) are the same
    for columnReal, columnGen in zip(real_feats.columns, features_for_the_first_beta):
        assert columnReal == columnGen

## small fig

In [2]:
cols_to_plot = [
    "cell_shape_volume",
    "cell_shape_surface_area",
    "dna_shape_surface_area",
    "dna_shape_volume",
#     'cell_inten_intensity_mean',
#     'cell_inten_intensity_median',
#     'cell_inten_intensity_std',
#     'dna_inten_intensity_mean',
#     'dna_inten_intensity_median',
#     'dna_inten_intensity_std'
]
betas_to_plot = [b for i,b in enumerate(betas) if i in [0,12,24]]

NameError: name 'betas' is not defined

In [138]:
# i=0 #unnormalized
i=1 #normalized

df = pd.DataFrame(columns=["beta", "type", *cols_to_plot])
for beta in betas_to_plot:
    df_gen = feature_dict[i]["gen"][beta][cols_to_plot].copy()
    df_gen["beta"] = beta
    df_gen["type"] = "generated"
    
    df_real = feature_dict[i]["real"][cols_to_plot].copy()
    df_real["beta"] = beta
    df_real["type"] = "real"
        
    df = df.append(df_gen)
    df = df.append(df_real)
    
df = df.reset_index(drop=True)

In [182]:
df_hist = pd.DataFrame(columns=["beta", "type"]+cols_to_plot+[f"{c}_density" for c in cols_to_plot])

n_bins=100
bin_dict = {col: np.histogram(df_gen[col], bins=n_bins)[1] for col in cols_to_plot}
bin_dict_centers = {col: (bins[1:] + bins[:-1])/2 for col,bins in bin_dict.items()}


for beta in betas_to_plot:
    df_gen_beta = df[(df["beta"]==beta) & (df["type"]=="generated")]
    df_gen_hist = pd.DataFrame(columns = df_hist.columns)
    for col in cols_to_plot:
        df_gen_hist[f"{col}_density"], _ = np.histogram(df_gen_beta[col], bins=bin_dict[col])
        df_gen_hist[col] = bin_dict_centers[col]
    df_gen_hist["beta"] = beta
    df_gen_hist["type"] = "generated"
    
    df_real_beta = df[(df["beta"]==beta) & (df["type"]=="real")]
    df_real_hist = pd.DataFrame(columns = df_hist.columns)
    for col in cols_to_plot:
        df_real_hist[f"{col}_density"], _ = np.histogram(df_real_beta[col], bins=bin_dict[col])
        df_real_hist[col] = bin_dict_centers[col]
    df_real_hist["beta"] = beta
    df_real_hist["type"] = "real"
    
    df_hist = df_hist.append(df_gen_hist)
    df_hist = df_hist.append(df_real_hist)
    
df_hist = df_hist.reset_index(drop=True)
# df_hist = df_hist.melt(id_vars=["beta","type"], var_name="feature", value_name="density")

In [221]:
df_hist = pd.DataFrame(columns=["beta", "type", "feature", "value", "density"])

n_bins=50
bin_dict = {col: np.histogram(df_gen[col], bins=n_bins)[1] for col in cols_to_plot}
bin_dict_centers = {col: (bins[1:] + bins[:-1])/2 for col,bins in bin_dict.items()}


for beta in betas_to_plot:
    df_gen_beta = df[(df["beta"]==beta) & (df["type"]=="generated")]
    df_gen_hist = pd.DataFrame(columns = df_hist.columns)
    for col in cols_to_plot:
        df_feat = pd.DataFrame(columns = df_hist.columns)
        df_feat["value"] = bin_dict_centers[col]
        df_feat["density"], _ = np.histogram(df_gen_beta[col], bins=bin_dict[col], density=True)
        df_feat["feature"] = col
        df_feat["beta"] = beta
        df_feat["type"] = "generated"
        df_gen_hist = df_gen_hist.append(df_feat)
    
    df_real_beta = df[(df["beta"]==beta) & (df["type"]=="real")]
    df_real_hist = pd.DataFrame(columns = df_hist.columns)
    for col in cols_to_plot:
        df_feat = pd.DataFrame(columns = df_hist.columns)
        df_feat["value"] = bin_dict_centers[col]
        df_feat["density"], _ = np.histogram(df_real_beta[col], bins=bin_dict[col], density=True)
        df_feat["feature"] = col
        df_feat["beta"] = beta
        df_feat["type"] = "real"
        df_real_hist = df_real_hist.append(df_feat)
    
    df_hist = df_hist.append(df_gen_hist)
    df_hist = df_hist.append(df_real_hist)
    
df_hist = df_hist.reset_index(drop=True)

/home/rorydm/miniconda3/envs/cfd2/lib/python3.7/site-packages/numpy/lib/histograms.py:898: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges


In [222]:
alt.Chart(df_hist).mark_area(
    opacity=0.5,
    interpolate='step'
).encode(
    alt.X("value:Q"),
    alt.Y("density:Q", stack=None),
    alt.Color('type:N', scale=alt.Scale(scheme='set1'))
).properties(
    height=200,
    width=200
).facet(
    column="beta:N",
    row="feature:N",
).resolve_scale(
    x='independent',
    y='independent'
)

alt.FacetChart(...)

In [226]:
df[(df["beta"] == 0.01)]["cell_shape_volume"]

0       4922
1       5203
2       5509
3       4673
4       5330
        ... 
9485    2930
9486    2930
9487    2930
9488    2930
9489    2930
Name: cell_shape_volume, Length: 9490, dtype: object

In [107]:
charts = {}

for col in cols_to_plot:
    charts[col] = alt.Chart(df).mark_area(
        opacity=0.3,
        interpolate='step'
    ).encode(
        alt.X(f"{col}:Q", bin=alt.Bin(maxbins=100)),
        alt.Y('count()', stack=None),
        alt.Color('type:N', scale=alt.Scale(scheme='set1'))
    ).properties(
        height=200,
        width=200
    ).facet(
        column="beta:N",
    ).resolve_scale(
        x='independent',
        y='independent'
    )

In [108]:
for col, chart in charts.items():
    display(chart)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

## large fig

In [99]:
cols_to_plot = [
    "cell_shape_volume",
    "cell_shape_surface_area",
    "dna_shape_surface_area",
    "dna_shape_volume",
    'cell_inten_intensity_mean',
#     'cell_inten_intensity_median',
    'cell_inten_intensity_std',
    'dna_inten_intensity_mean',
#     'dna_inten_intensity_median',
    'dna_inten_intensity_std'
]
betas_to_plot = [b for i,b in enumerate(betas) if i in [0,2,7,12,17,22,24]]

In [100]:
i=1
intensity_norms = ['unnormalized', 'normalized']

df = pd.DataFrame(columns=["beta", *cols_to_plot])

for beta in betas_to_plot:
    df_gen = feature_dict[i]["gen"][beta][cols_to_plot].copy()
    df_gen["beta"] = beta
    df_gen["type"] = "generated"
    
    df_real = feature_dict[i]["real"][cols_to_plot].copy()
    df_real["beta"] = beta
    df_real["type"] = "real"
        
    df = df.append(df_gen)
    df = df.append(df_real)
    
df = df.reset_index(drop=True)

In [101]:
day_18_color = "#0098EA"
day_32_color = "#FFB2FF"
day_colors = [day_18_color, day_32_color]

day_color_scale = alt.Scale(range=day_colors)

In [102]:
charts = {}

for col in cols_to_plot:
    charts[col] = alt.Chart(df).mark_area(
        opacity=0.3,
        interpolate='step'
    ).encode(
        alt.X(f"{col}:Q", bin=alt.Bin(maxbins=100)),
        alt.Y('count()', stack=None),
        alt.Color('type:N', scale=alt.Scale(scheme='set1'))
    ).properties(
        height=200,
        width=200
    ).facet(
        column="beta:N",
    ).resolve_scale(
        x='independent',
        y='independent'
    )

In [103]:
for col, chart in charts.items():
    display(chart)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)

alt.FacetChart(...)